# Using the OpenVINO™ Execution Provider for YOLOv8 Detection

[Source (modified from Microsoft ONNX Runtime OpenVINO EP Examples)](https://github.com/microsoft/onnxruntime-inference-examples/tree/main/python/OpenVINO_EP/yolov8_object_detection)

In [6]:
!pip install ultralytics


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
#Defining a sample image for inference
image_url = "https://ultralytics.com/images/bus.jpg" 

other images options : https://storage.openvinotoolkit.org/data/test_data/images/cat.jpg, https://storage.googleapis.com/download.tensorflow.org/example_images/grace_hopper.jpg


## Importing Required Packages

In [4]:
import os
import requests
import shutil
import cv2
import numpy as np
import onnxruntime as rt
import torch
from statistics import mean
from datetime import datetime
from ultralytics import YOLO
from ultralytics.data.augment import LetterBox

In [5]:
# Parameters for pre-processing
imgsz = (640,640) # default value for this usecase.
stride = 32 # default value for this usecase( differs based on the model selected

print(image_url)
def preprocess(image_url):
    ## Set up the image URL
    path = os.getcwd()
    image_path=os.path.join(path, image_url.split("/")[-1])
    # Open the url image, set stream to True, this will return the stream content.
    r = requests.get(image_url, stream = True)
    # Check if the image was retrieved successfully
    if r.status_code == 200:
        # Set decode_content value to True, otherwise the downloaded image file's size will be zero.
        r.raw.decode_content = True
        # Open a local file with wb ( write binary ) permission.
        with open(image_path,'wb') as f:
            shutil.copyfileobj(r.raw, f)
        print('Image sucessfully downloaded: ',path)
    else:
        print('Image couldn\'t be retreived')
        return
    image_abs_path = os.path.abspath(image_path)
    if os.path.isfile(image_abs_path) and image_abs_path.split('.')[-1].lower() in ['jpg', 'jpeg', 'png']:
        # Load Image
        img0 = cv2.imread(image_abs_path)
        # Padded resize
        #Letterbox: Resize image and padding for detection, instance segmentation, pose
        img = LetterBox(imgsz, stride=stride)(image=img0.copy())
        # Convert
        img =  img.transpose((2, 0, 1))[::-1]  # BGR to RGB, to 3x416x416
        img = np.ascontiguousarray(img)
        img = img.astype(np.float32)  # uint8 to fp16/32
        img /= 255.0  # 0 - 255 to 0.0 - 1.0
        if img.ndim == 3:
            img = np.expand_dims(img, axis=0)
        return img0, img
    else:
        print("Invalid image format.")
        return

org_input, model_input = preprocess(image_url)

https://ultralytics.com/images/bus.jpg
Image sucessfully downloaded:  C:\Users\riach\Downloads


## Downloading a YOLOv8 Model and Exporting it to Onnx

In [6]:
from ultralytics import YOLO

# Load the YOLOv8 model
model = YOLO("yolov8n.pt")

# Export the model to ONNX format
model.export(format="onnx")  # creates 'yolov8n.onnx'

# Load the exported ONNX model
onnx_model = YOLO("yolov8n.onnx")

# Run inference
results = onnx_model("https://ultralytics.com/images/bus.jpg")

Ultralytics YOLOv8.2.81  Python-3.11.9 torch-2.2.2+cpu CPU (Intel Core(TM) Ultra 9 185H)
YOLOv8n summary (fused): 168 layers, 3,151,904 parameters, 0 gradients, 8.7 GFLOPs

PyTorch: starting from 'yolov8n.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 84, 8400) (6.2 MB)

ONNX: starting export with onnx 1.16.0 opset 17...
ONNX: export success  0.6s, saved as 'yolov8n.onnx' (12.2 MB)

Export complete (2.3s)
Results saved to C:\Users\riach\Downloads
Predict:         yolo predict task=detect model=yolov8n.onnx imgsz=640  
Validate:        yolo val task=detect model=yolov8n.onnx imgsz=640 data=coco.yaml  
Visualize:       https://netron.app
WARNING  Unable to automatically guess model task, assuming 'task=detect'. Explicitly define task for your model, i.e. 'task=detect', 'segment', 'classify','pose' or 'obb'.
Loading yolov8n.onnx for ONNX Runtime inference...

Found https://ultralytics.com/images/bus.jpg locally at bus.jpg
image 1/1 C:\Users\riach\Downloads\bus.jpg: 640

## Initialization

In [8]:
original_model_path = "/content/model/yolov8m.onnx"

def initialize(quantize=False, device='OVEP'):
    "Initialize the model also getting model output and input names"
    initialized = True
    model_dir = os.getcwd()
    ov_model = None; mlas_model = None
    so = rt.SessionOptions()
    if device == 'OVEP':
        if quantize == True:
            print("Inferencing through OVEP")
            ov_model = rt.InferenceSession(quantized_model_path, so,
                                       providers=['OpenVINOExecutionProvider'],
                                       provider_options=[{'device_type' : 'CPU_FP32'}])
        else:
            ov_model = rt.InferenceSession(original_model_path, so,
                                       providers=['OpenVINOExecutionProvider'],
                                        provider_options=[{'device_type' : 'CPU_FP32'}])
    elif device == 'CPUEP':
        if quantize == True:
            mlas_model = rt.InferenceSession(quantized_model_path, so, providers=['CPUExecutionProvider'])
        else:
            mlas_model = rt.InferenceSession(original_model_path, so, providers=['CPUExecutionProvider'])

    if device == 'OVEP':
      input_names = ov_model.get_inputs()[0].name
      outputs = ov_model.get_outputs()
    else:
      input_names = mlas_model.get_inputs()[0].name
      outputs = mlas_model.get_outputs()
    output_names = list(map(lambda output:output.name, outputs))
    return input_names, output_names, mlas_model, ov_model

device = 'OVEP'
input_names, output_names, mlas_model, ov_model = initialize(device=device)

## Inference

In [9]:
#Select number of iterations for inference

no_of_iterations = 20
warmup_iterations = 10

In [10]:
inf_lst = []
def inference(input_names, output_names, device, mlas_model, ovep_model, model_input):
    if device == 'CPUEP':
        print("Performing ONNX Runtime Inference with default CPU EP.")
        for i in range(no_of_iterations):
          start_time = datetime.now()
          prediction = mlas_model.run(output_names, {input_names: model_input})
          end_time = datetime.now()
          # print((end_time - start_time).total_seconds())
          if i > warmup_iterations:
            inf_lst.append((end_time - start_time).total_seconds())
    elif device == 'OVEP':
        print("Performing ONNX Runtime Inference with OpenVINO EP.")
        for i in range(no_of_iterations):
          start_time = datetime.now()
          prediction = ovep_model.run(output_names, {input_names: model_input})
          end_time = datetime.now()
          # print((end_time - start_time).total_seconds())
          if i > warmup_iterations:
            inf_lst.append((end_time - start_time).total_seconds())
    else:
        print("Invalid Device Option. Supported device options are 'cpu', 'CPU_FP32'.")
        return None
    return prediction, (end_time - start_time).total_seconds()

inference_output = inference(input_names, output_names, device, mlas_model, ov_model, model_input)
average_inference_time = np.average(inf_lst)
print(f'Average inference time is for {no_of_iterations - warmup_iterations} iterations is {average_inference_time} sec')

Performing ONNX Runtime Inference with OpenVINO EP.
Average inference time is for 10 iterations is 0.022719 sec


## Final Inference on Webcam Input

In [ ]:
from ultralytics import YOLO
from ultralytics.solutions import ai_gym
import cv2

model = YOLO("yolov8n.onnx")
cap = cv2.VideoCapture(0)
assert cap.isOpened(), "Error reading video file"
w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))
frame_count = 0
while cap.isOpened():
    success, im0 = cap.read()
    if not success:
      print("Video frame is empty or video processing has been successfully completed.")
      break
    frame_count += 1
    results = model.predict(im0)  # Prediction also supported
    res_plotted = results[0].plot()
    cv2.imshow("Tracking_Stream", res_plotted)
    key = cv2.waitKey(1)
    if key == ord('q'):
        break

cv2.destroyAllWindows()
cap.release() #Release video sources